In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession \
    .builder \
    .appName("Learning Spark") \
    .getOrCreate()

df = spark.read.json("harvester/occ/tests/eduardofv-air-2.local-sneaky-20230627_200714.jsonl.gz")
df

DataFrame[identifier: string, jobposting: string, search_engine_type: string, url: string, version: string]

In [2]:
df.show()

+----------+--------------------+------------------+--------------------+-------+
|identifier|          jobposting|search_engine_type|                 url|version|
+----------+--------------------+------------------+--------------------+-------+
|  16958865|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958862|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958871|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958873|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958878|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958879|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16958880|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16676692|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16943940|{"props":{"pagePr...|               JDP|https://www.occ.c...|  0.0.1|
|  16115833|{"pr

In [9]:
import json
json.loads(df.select("jobposting").first()["jobposting"])

{'props': {'pageProps': {'initialApolloState': {'CountryLocation:MX': {'__typename': 'CountryLocation',
     'id': 'MX',
     'description': 'México'},
    'JobLevel:': {'__typename': 'JobLevel', 'id': '', 'url': None},
    'JobCategory:17': {'__typename': 'JobCategory',
     'description': 'Tecnologías de la Información - Sistemas',
     'id': '17',
     'url': 'empleos/trabajo-en-tecnologias-de-la-informacion-sistemas/',
     'rel': None},
    'JobSubcategory:416': {'__typename': 'JobSubcategory',
     'id': '416',
     'description': 'Big data',
     'url': 'empleos/trabajo-en-tecnologias-de-la-informacion-sistemas-big-data/'},
    'JobRequisition:': {'__typename': 'JobRequisition',
     'id': '',
     'hasIdRequisicion': False},
    'Job:16958865': {'__typename': 'Job',
     'id': '16958865',
     'url': '/empleo/oferta/16958865-consultor-arquitectura-de-datos?rank=1&page=1&sessionid=&userid=&uuid=5a545ad1-7905-442f-862b-51409e2b4ad7&origin=unknown&type=3&ai=false&ais=&showseo=true

In [11]:
with open("occ_jobposting_sample.json", "w") as file:
    file.write(df.select("jobposting").first()["jobposting"])

In [12]:
jsonobj = spark.read.json("occ_jobposting_sample.json", multiLine=True)
schema = jsonobj.schema
schema

StructType([StructField('assetPrefix', StringType(), True), StructField('buildId', StringType(), True), StructField('dynamicIds', ArrayType(LongType(), True), True), StructField('gssp', BooleanType(), True), StructField('isFallback', BooleanType(), True), StructField('page', StringType(), True), StructField('props', StructType([StructField('__N_SSP', BooleanType(), True), StructField('pageProps', StructType([StructField('applyFromEmail', StringType(), True), StructField('initialApolloState', StructType([StructField('CountryLocation:MX', StructType([StructField('__typename', StringType(), True), StructField('description', StringType(), True), StructField('id', StringType(), True)]), True), StructField('Job:16958865', StructType([StructField('__typename', StringType(), True), StructField('applied', BooleanType(), True), StructField('autoinclusion', StructType([StructField('__typename', StringType(), True), StructField('autoinclusion', BooleanType(), True), StructField('source', StringTyp

In [14]:
dfexp = df.withColumn("jp", F.from_json(F.col("jobposting"), schema))
dfexp.printSchema()

root
 |-- identifier: string (nullable = true)
 |-- jobposting: string (nullable = true)
 |-- search_engine_type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- version: string (nullable = true)
 |-- jp: struct (nullable = true)
 |    |-- assetPrefix: string (nullable = true)
 |    |-- buildId: string (nullable = true)
 |    |-- dynamicIds: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- gssp: boolean (nullable = true)
 |    |-- isFallback: boolean (nullable = true)
 |    |-- page: string (nullable = true)
 |    |-- props: struct (nullable = true)
 |    |    |-- __N_SSP: boolean (nullable = true)
 |    |    |-- pageProps: struct (nullable = true)
 |    |    |    |-- applyFromEmail: string (nullable = true)
 |    |    |    |-- initialApolloState: struct (nullable = true)
 |    |    |    |    |-- CountryLocation:MX: struct (nullable = true)
 |    |    |    |    |    |-- __typename: string (nullable = true)
 |    |    |    |    |    |

In [21]:
dfexp.select("jp.props.pageProps.jobId").show()

+--------+
|   jobId|
+--------+
|16958865|
|16958862|
|16958871|
|16958873|
|16958878|
|16958879|
|16958880|
|16676692|
|16943940|
|16115833|
|16610287|
|16958889|
|16895564|
|16958890|
|16739939|
|15694816|
|16958894|
|16958765|
|16958772|
|16958774|
+--------+
only showing top 20 rows

